***Warnings***
- rule_by_efficient_apriori() has some question, why the 'if' can save the variable temp_set

In [2]:
import pandas as pd
import time

In [3]:
data = pd.read_csv('BreadBasket_DMS.csv')

In [4]:
data.head(8)

,Date,Time,Transaction,Item
0,2016/10/30,9:58:11,1,NONE
1,2016/10/30,10:05:34,2,Scandinavian
2,2016/10/30,10:05:34,2,Scandinavian
3,2016/10/30,10:07:57,3,Hot chocolate
4,2016/10/30,10:07:57,3,Jam
5,2016/10/30,10:07:57,3,Cookies
6,2016/10/30,10:08:41,4,Muffin
7,2016/10/30,10:13:03,5,Coffee


In [5]:
data['Item'] = data['Item'].str.lower()

In [6]:
data.head()

,Date,Time,Transaction,Item
0,2016/10/30,9:58:11,1,none
1,2016/10/30,10:05:34,2,scandinavian
2,2016/10/30,10:05:34,2,scandinavian
3,2016/10/30,10:07:57,3,hot chocolate
4,2016/10/30,10:07:57,3,jam


In [7]:
data[data.Item == 'none'].shape[0] / data.shape[0]

0.036960503451838635

In [8]:
data = data.drop(data[data.Item == 'none'].index)

In [9]:
def rule_by_efficient_apriori():
    from efficient_apriori import apriori
    start = time.time()
    orders_series = data.set_index('Transaction')['Item']
    transactions = []
    temp_idx = -1
    # MB, In python language, the set is object, u can change the state of object. 
    for idx, val in orders_series.items():
        if idx != temp_idx:
            temp_set = set()
            temp_idx = idx
            temp_set.add(val)
            transactions.append(temp_set)
        else:  # the val haven't been added the set()
            temp_set.add(val)
    itemsets, rules = apriori(transactions, min_support=0.02, min_confidence=0.5)
    print('freq itemsets', itemsets)
    print('association rules', rules)
    end = time.time()
    print('time is ', end - start)
#     print('count of ',cnt)
    return transactions

In [10]:
transactions = rule_by_efficient_apriori()

freq itemsets {1: {('alfajores',): 344, ('bread',): 3096, ('brownie',): 379, ('cake',): 983, ('coffee',): 4528, ('cookies',): 515, ('farm house',): 371, ('hot chocolate',): 552, ('juice',): 365, ('medialuna',): 585, ('muffin',): 364, ('pastry',): 815, ('sandwich',): 680, ('scandinavian',): 275, ('scone',): 327, ('soup',): 326, ('tea',): 1350, ('toast',): 318, ('truffles',): 192}, 2: {('bread', 'cake'): 221, ('bread', 'coffee'): 852, ('bread', 'pastry'): 276, ('bread', 'tea'): 266, ('cake', 'coffee'): 518, ('cake', 'tea'): 225, ('coffee', 'cookies'): 267, ('coffee', 'hot chocolate'): 280, ('coffee', 'juice'): 195, ('coffee', 'medialuna'): 333, ('coffee', 'pastry'): 450, ('coffee', 'sandwich'): 362, ('coffee', 'tea'): 472, ('coffee', 'toast'): 224}}
association rules [{cake} -> {coffee}, {cookies} -> {coffee}, {hot chocolate} -> {coffee}, {juice} -> {coffee}, {medialuna} -> {coffee}, {pastry} -> {coffee}, {sandwich} -> {coffee}, {toast} -> {coffee}]
time is  0.34378862380981445


In [11]:
def encode_units(val):
    if val <= 0:
        return 0
    if val >= 1:
        return 1
def rule_by_mlxtend():
    from mlxtend.frequent_patterns import apriori
    from mlxtend.frequent_patterns import association_rules
    pd.options.display.max_columns = 100
    start = time.time()
    hot_encoded_df = data.groupby(['Transaction', 'Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
    hot_encoded_df = hot_encoded_df.applymap(encode_units)
    frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.5)
    print('freq itemset', frequent_itemsets)
    print('association rules', rules[(rules['lift'] >= 1) & (rules['confidence'] >= 0.5)])
    end = time.time()
    print('Time: ', end-start)

In [12]:
x = data.groupby(['Transaction', 'Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')

In [13]:
x.applymap(encode_units)

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
rule_by_mlxtend()

freq itemset      support                 itemsets
0   0.036348              (alfajores)
1   0.327134                  (bread)
2   0.040046                (brownie)
3   0.103867                   (cake)
4   0.478445                 (coffee)
5   0.054417                (cookies)
6   0.039201             (farm house)
7   0.058326          (hot chocolate)
8   0.038567                  (juice)
9   0.061813              (medialuna)
10  0.038462                 (muffin)
11  0.086116                 (pastry)
12  0.071851               (sandwich)
13  0.029057           (scandinavian)
14  0.034552                  (scone)
15  0.034446                   (soup)
16  0.142646                    (tea)
17  0.033601                  (toast)
18  0.020287               (truffles)
19  0.023352            (cake, bread)
20  0.090025          (coffee, bread)
21  0.029163          (pastry, bread)
22  0.028107             (tea, bread)
23  0.054734           (cake, coffee)
24  0.023774              (cake, tea)